## Efficient Dataset Condensation with Synthetic Data Parameterization

### Get the Dataset and basic preprocessing to remove skewness of test set

### For the standard dataset, run this:

In [1]:
%%bash
wget https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part3_Training_Data.zip
unzip  ISBI2016_ISIC_Part3_Training_Data.zip
mv ISBI2016_ISIC_Part3_Training_Data ISIC2016_TrainingImages

wget https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part3_Test_Data.zip
unzip  ISBI2016_ISIC_Part3_Test_Data.zip
mv ISBI2016_ISIC_Part3_Test_Data ISIC2016_TestImages

### For cropped dataset from binary masks, run this instead:

In [ ]:
%%bash
wget https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part3B_Training_Data.zip
unzip  ISBI2016_ISIC_Part3B_Training_Data.zip
python3 preprocess.py --source_dir "./ISBI2016_ISIC_Part3B_Training_Data" --dest_dir "./ISIC2016_TrainingImages" --img_size 150 150
rm -rf ISBI2016_ISIC_Part3B_Training_Data

wget https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part3B_Test_Data.zip
unzip  ISBI2016_ISIC_Part3B_Test_Data.zip
python3 preprocess.py --source_dir "./ISBI2016_ISIC_Part3B_Test_Data" --dest_dir "./ISIC2016_TestImages" --img_size 150 150
rm -rf ISBI2016_ISIC_Part3_Test_Data

### Now let's get the labels and preprocess them

In [ ]:
%%bash
wget https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part3_Training_GroundTruth.csv
wget https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part3_Test_GroundTruth.csv
mv ISBI2016_ISIC_Part3_Training_GroundTruth.csv ISIC2016_TrainingLabels.csv
mv ISBI2016_ISIC_Part3_Test_GroundTruth.csv ISIC2016_TestLabels.csv

In [ ]:
import csv
import pandas as pd
import random

path_test_csv = "./ISIC2016_TestLabels.csv"
df = pd.read_csv(path_test_csv, header=None)
indices = df.index[df.iloc[:, 1] == 0.0].tolist()
print(indices, len(indices))
indices_remove = random.sample(indices, 229)
print(indices_remove, len(indices_remove))
print(df)
df.drop(df.index[indices_remove], inplace=True)
print(df)
print(df.shape)
print(df.iloc[:, 1].value_counts())
df.to_csv(path_test_csv, index=False, header=None)

### Install required packages

In [ ]:
%%bash
pip install efficientnet_pytorch

### Run the condensation

In [ ]:
%%bash
python Efficient-Dataset-Condensation/condense.py -d isic2016 --data_dir "../" --ipc 10 --verbose -n convnet --epochs 100 --niter 250 --inner_loop 100 --n_data 200